In [1]:
import pandas as pd
import re

I import the subset from ZINC15 taken from Chris' dataset on the rostserver: "canonical_smiles.txt". It conains 100 canonical SMILES.

In [2]:
subset = pd.read_table('../datasets/sets_from_Chris/canonical_smiles_first_100.txt', header=None)

subset = subset.drop_duplicates()          # throw out duplicates

In [3]:
subset = subset.rename(columns={0: "canonical_SMILES"})

- Wofür steht das @? \
In SMILES, tetrahedral centers may be indicated by a simplified chiral specification (@ or @@) written as an atomic property following the atomic symbol of the chiral atom.\
The symbol "@" indicates that the following neighbors are listed anticlockwise (it is a "visual mnemonic" in that the symbol looks like an anticlockwise spiral around a central circle).\
"@@" indicates that the neighbors are listed clockwise (you guessed it, anti-anti-clockwise). 


- Wurde die Transformation mit RDKit gemacht?

# Find out the alphabet

Are we interested in the non-alphabetic signs? I can't think of any.

In [4]:
total_SMILES = "".join(subset.canonical_SMILES)

In [5]:
total_SMILES

'C[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@H]5OC(=O)[C@@]3(O[C@H]3OC(=O)[C@H](O)[C@]34[C@H](C(C)(C)C)[C@H]5OS(=O)(=O)C(F)(F)F)[C@@]12OC[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@@H]5OC(=O)[C@@]3(O[C@H]3OC(=O)[C@H](O)[C@]34[C@H](C(C)(C)C)[C@H]5OS(=O)(=O)C(F)(F)F)[C@@]12OC[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@H]5OC(=O)[C@@]3(O[C@H]3OC(=O)[C@H](O)[C@@]34[C@H](C(C)(C)C)[C@H]5OS(=O)(=O)C(F)(F)F)[C@@]12OC[C@@H]1C(=O)O[C@H]2[C@H](O)[C@@]34[C@@H]5OC(=O)[C@@]3(O[C@H]3OC(=O)[C@H](O)[C@@]34[C@H](C(C)(C)C)[C@H]5OS(=O)(=O)C(F)(F)F)[C@@]12OO=c1cc(-c2ccc(O)cc2)oc2c(O)c(O[C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O[C@@H]3O[C@H](CO)[C@@H](O)[C@@H](O)[C@H]3O)cc(O)c12COc1cc2oc(-c3ccc(O)cc3)cc(=O)c2c(O)c1[C@@H]1O[C@H](CO)[C@@H](O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)[C@H](O)[C@H]1OO=c1cc(-c2ccc(O)cc2)oc2c([C@@H]3O[C@H](CO)[C@H](O)[C@H](O)[C@H]3O)c(O)c([C@@H]3O[C@H](CO)[C@H](O)[C@H](O)[C@H]3O)c(O)c12COc1cc2oc(-c3ccc(O)cc3)cc(=O)c2c(O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)c1[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@H]1

There are lower-case letters because:
- The second letter of two-character symbols must be entered in lower case.
- Atoms in aromatic rings are specified by lower case letters, e.g., aliphatic carbon is represented by the capital letter C, aromatic carbon by lower case c.

=> I must find also 2-letter elements!

In [6]:
# define an empty alphabet
alphabet = set([])

### 2-letter chemical elements

In [7]:
two_letter_elements = re.findall("[A-Z][a-z]", total_SMILES)
two_letter_elements = sorted(set(two_letter_elements))

In [8]:
#two_letter_elements

In [9]:
periodic_table_2_letter = ['He',
                           'Li', 'Be', 'Ne',
                           'Na', 'Mg', 'Al', 'Si', 'Cl', 'Ar',
                           'Ca', 'Sc', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
                           'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'Xe',
                           'Cs', 'Ba', 'Hf', 'Ta', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn',
                           'Fr', 'Ra', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og',
                           'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu',
                           'Ac', 'Th', 'Pa', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr']

In [10]:
for element in two_letter_elements:
    if element in periodic_table_2_letter:
        
        # Add the 2-letter elements which are in the periodic table
        alphabet.add(element)
        
        # Take the added 2-letter elements out of my search string (SMILES)
        total_SMILES = re.sub(element, "", total_SMILES) 

In [11]:
#alphabet

### 1-letter chemical elements

In [12]:
one_letter_elements = re.findall("[A-Z]|[a-z]", total_SMILES)
one_letter_elements = sorted(set(one_letter_elements))

In [13]:
one_letter_elements

['C', 'F', 'H', 'N', 'O', 'P', 'S', 'c', 'n', 'o', 's']

In [14]:
alphabet.update(one_letter_elements)

for element in one_letter_elements:    
    total_SMILES = re.sub(element, "", total_SMILES)         

### Rest

Falls ich nur 2-letter chemical Elements als Elemente mit >1 Stelle habe, kann ich 1-letter chemical elements und "Rest" zusammenschmeissen.

In [15]:
rest = set(list(total_SMILES))
alphabet.update(rest)

In [16]:
alphabet

{'#',
 '(',
 ')',
 '-',
 '/',
 '1',
 '2',
 '3',
 '4',
 '5',
 '=',
 '@',
 'C',
 'Cl',
 'F',
 'H',
 'N',
 'O',
 'P',
 'S',
 'Si',
 '[',
 ']',
 'c',
 'n',
 'o',
 's'}

### Formal charge

In [17]:
total_SMILES = "".join(subset.canonical_SMILES)    # retrieve the original SMILES string
"+" in total_SMILES # False

False

In the subset I have right now, there is no +\
=> There is probably no charge indicated!\
-> Is that always the case for canonical SMILES? (on the daylight page, it seems so)\
\
Check also with PubChem Set!

### Implicit hydrogen count

In this subset, I also can't find an example of that.\
e.g. h5

### Degree

Also not present in this subset

### Valence

Also not present in this subset

## Write output

In [30]:
#with open("alphabet_result.txt", "w") as output_file:
#    output_file.writelines(list(alphabet))